In [68]:
# Juego de ultimatum
# Autor: Sara Brborich
# Fecha: 11/4/2023
# Fecha de modificación: 18/4/2023

import random

class agent:

    #constructor
    def __init__(self, identifier, role): 
        self.id = identifier 
        self.role = role
        self.wealth = 0
        self.endowment = 10 # lo que cada jugador tiene al principio
        self.propSplit = random.randint(1, self.endowment)  # player 1 proposed split
        self.minSplit= random.randint(1,self.endowment) #player 2 minimum split
        self.played=False #indica si el jugador ya jugó o no
        self.prop_Split_Strategy_Vector = list(range(1,self.endowment+1)) #vector de estrategias de player 1
        self.min_Split_Strategy_Vector = list(range(1,self.endowment+1)) #vector de estrategias de player 2
        
    def __str__(self):
        lines=[]
        lines.append("*********************")
        lines.append("Agent ID: " + str(self.id) )
        if(self.role==1):
            lines.append("Proposed split: " + str(self.propSplit))
        if(self.role==2):
            lines.append("Minimum split: " + str(self.minSplit))
        lines.append("Wealth: " + str(self.wealth))
        lines.append(" ")
        return '\n'.join(lines)
    

    def __eq__(self, other):
        if self.id == other.id:
            return True
        else:
            return False


    def __hash__(self): #devuelve el id del agente
        return self.id        


    def setRLStrategy(self):
        if self.role == 1:
            self.propSplit = random.choice(self.prop_Split_Strategy_Vector) #selecciona una estrategia al azar
        if self.role == 2:
            self.minSplit = random.choice(self.min_Split_Strategy_Vector) #selecciona una estrategia al azar

    def updateRLStrategy(self, strat):
        if self.role == 1:
            self.prop_Split_Strategy_Vector.append(strat) #actualiza el vector de estrategias
        if self.role == 2:
            self.min_Split_Strategy_Vector.append(strat)

    


    # Funcion que evalua el split de los jugadores
    # player 2 check if player 1 proposed split is greater than minimum split

    def evaluateSplit(self, partner):
        #print("Player 1 offered split: " + str(self.propSplit))
        #print("Player 2 minimum split: " + str(partner.minSplit))
        if partner.propSplit >= partner.minSplit:
            print("\nSplit accepted")
            
            #actualizar la riqueza de los jugadores            
            self.wealth = self.wealth + partner.propSplit
            partner.wealth = partner.wealth + (10 - partner.propSplit) 
            
            print("\nPlayer 1 wealth: " + str(self.wealth))
            print("Player 2 wealth: " + str(partner.wealth))

            #actualizar el vector de estrategias
            self.updateRLStrategy(self.propSplit) 
            partner.updateRLStrategy(partner.propSplit)
            
        else:
            print("Split rejected")

        self.setPlayed(True) #indica que el jugador ya jugó
        partner.setPlayed(True) 
    


    def setPlayed(self, p): #indica si el jugador ya jugó o no
        self.played = p 
    


    

    

In [69]:
class Model: # modelo de mundo con muchos agentes que jueguen el juego de ultimatum

    def __init__(self, num_agents, num_iterations):
        self.num_agents = num_agents
        self.num_iterations= num_iterations
        self.agents =[] # list of agents 
        self.setupAgents() #crea los agentes y los agrega a la lista de agentes 


    def setupAgents(self): #crea los agentes
        for i in range(self.num_agents):
            self.agents.append(agent(i,2))

    def assignRole(self, partners): #asigna roles al azar
        if random.random() < 0.5: #50% de probabilidad de que el primer jugador sea el player 1
            partners[0].role = 1 
            partners[1].role = 2
            return partners[0], partners[1]
        
        else: #50% de probabilidad de que el primer jugador sea el player 2
            partners[0].role = 2
            partners[1].role = 1
            return partners[1], partners[0]
 

    def execute(self):
        for t in range(self.num_iterations): #iteraciones

            print("@@@@@@@@@@@@@@@@@@@@@@@@@@")
            print("Iteration: " + str(t))
            print("@@@@@@@@@@@@@@@@@@@@@@@@@@")


            agents_Not_Played=list(filter(lambda x: x.played==False, self.agents)) #lista de agentes que no jugaron todavía
            
            while len(agents_Not_Played)>1:                
                partners = random.sample(agents_Not_Played, 2) #seleccionar dos agentes al azar

                print("\nAgents:")                
                for p in partners:                  
                    print(p)    

                #Asignar los roles de los jugadores al azar
                player1, player2 = self.assignRole(partners)
                
                #seleccionar una estrategia al azar para cada jugador                
                player1.setRLStrategy()                
                player2.setRLStrategy() 

                #evaluar el split de los jugadores                
                player2.evaluateSplit(player1) #evaluar el split de los jugadores

                agents_Not_Played=list(filter(lambda x: x.played==False, self.agents)) #lista de agentes que no jugaron todavía

            print("\n\nAgentes que no juegan todavía:")
            list(map(lambda x: x.setPlayed(False), self.agents)) #indica que todos los agentes no jugaron todavía

            print("@@@@@@@@@@@@@@@@@@@@@@@@@@")
            print("End of iteration")
            print("@@@@@@@@@@@@@@@@@@@@@@@@@@")    
            print("\n")    
        


In [70]:
m1=Model(10,2)
len(m1.agents)
m1.execute() #ejecutar el modelo



@@@@@@@@@@@@@@@@@@@@@@@@@@
Iteration: 0
@@@@@@@@@@@@@@@@@@@@@@@@@@

Agents:
*********************
Agent ID: 2
Minimum split: 10
Wealth: 0
 
*********************
Agent ID: 8
Minimum split: 1
Wealth: 0
 
Split rejected

Agents:
*********************
Agent ID: 9
Minimum split: 10
Wealth: 0
 
*********************
Agent ID: 0
Minimum split: 5
Wealth: 0
 

Split accepted

Player 1 wealth: 7
Player 2 wealth: 3

Agents:
*********************
Agent ID: 6
Minimum split: 4
Wealth: 0
 
*********************
Agent ID: 1
Minimum split: 10
Wealth: 0
 
Split rejected

Agents:
*********************
Agent ID: 3
Minimum split: 4
Wealth: 0
 
*********************
Agent ID: 7
Minimum split: 1
Wealth: 0
 

Split accepted

Player 1 wealth: 5
Player 2 wealth: 5

Agents:
*********************
Agent ID: 4
Minimum split: 9
Wealth: 0
 
*********************
Agent ID: 5
Minimum split: 3
Wealth: 0
 

Split accepted

Player 1 wealth: 10
Player 2 wealth: 0


Agentes que no juegan todavía:
@@@@@@@@@@@@@@@@@@@@@@@@@@